In [3]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [4]:
from traj_xai.src import SimpleTransformerModel, TrajFormerModel

/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
from pactus import Dataset

## Segmentation


In [6]:
from traj_xai.src import (
    mdl_segmentation,
    rdp_segmentation,
    sliding_window_segmentation,
)

## Pertubation


In [7]:
from traj_xai.src import (
    gaussian_perturbation,
    scaling_perturbation,
    rotation_perturbation,
)

## Evaluation


## Experiment with all trajectories


In [ ]:
# LSTM Model
SEED = 0

# datasets = [
#     # Dataset.geolife(),
#     Dataset.animals(), x
#     # Dataset.hurdat2(), x
#     Dataset.uci_characters(), x
#     Dataset.cma_bst(), x
#     # Dataset.mnist_stroke(), 1k x
#     # Dataset.uci_pen_digits(), x
#     # Dataset.uci_gotrack(), x
#     Dataset.uci_movement_libras(),
# ]
# train, test = dataset.mnist_stroke(.8, random_state=SEED)

# Load dataset
dataset = Dataset.uci_characters()

# Split data into train and test subsets
train, test = dataset.split(0.8, random_state=SEED)
# train, test = dataset.cut(69000)

In [9]:
train.trajs[0].r

Vector([[ 98.,  89.],
        [ 53., 100.],
        [ 12.,  72.],
        [  0.,  27.],
        [ 37.,   0.],
        [ 86.,  21.],
        [100.,  63.],
        [ 92.,  90.]])

In [10]:
# train.classes

### DL Model


In [19]:
max_length = 1000
embedding_size = 64

# Build and train the model
model = TrajFormerModel(
    c_out=len(dataset.classes),
)

# Train the model on the train dataset
model.train(train, dataset, epochs=10, batch_size=64)

# Evaluate the model on a test dataset
evaluation = model.evaluate(test)

# Print the evaluation
evaluation.show()

13:06:34 [INFO] Epoch 10/10, Loss: 89.1162
13:06:34 [INFO] Trained TrajFormer model with 69000 samples



General statistics:

Accuracy: 0.971
F1-score: 0.971
Mean precision: 0.971
Mean recall: 0.970

Confusion matrix:

0      1      2      3      4      5      6      7      8      9      precision
99.02  0.0    0.0    0.0    0.0    1.18   0.98   0.0    0.0    0.0    98.06  
0.0    98.32  0.0    0.0    0.0    0.0    1.96   0.0    0.0    0.0    98.32  
0.0    0.0    93.94  0.0    0.0    0.0    0.98   0.0    0.0    0.0    98.94  
0.0    0.0    2.02   98.04  0.0    1.18   0.0    0.0    0.0    0.0    97.09  
0.0    0.0    0.0    0.0    94.57  0.0    1.96   0.0    0.0    0.0    97.75  
0.0    0.0    0.0    0.98   0.0    96.47  0.98   0.0    0.0    0.0    97.62  
0.0    0.0    0.0    0.0    0.0    1.18   93.14  0.0    0.0    0.0    98.96  
0.0    1.68   2.02   0.0    2.17   0.0    0.0    100.0  0.0    1.11   94.26  
0.98   0.0    2.02   0.0    0.0    0.0    0.0    0.0    98.94  1.11   95.88  
0.0    0.0    0.0    0.98   3.26   0.0    0.0    0.0    1.06   97.78  94.62  
-------------------------

### ML Model


In [12]:
# # ML Model
# SEED = 0

# # datasets = [
# #     # Dataset.geolife(),
# #     Dataset.animals(),
# #     # Dataset.hurdat2(), -> RF cant work
# #     Dataset.cma_bst(),
# #     # Dataset.mnist_stroke(), -> okay
# #     # Dataset.uci_pen_digits(),
# #     # Dataset.uci_gotrack(),
# #     Dataset.uci_characters(),
# #     Dataset.uci_movement_libras(),-> okay
# # ]

# # Load dataset
# dataset = Dataset.geolife()

# # Split data into train and test subsets
# # train, test = dataset.split(.995, random_state=SEED)

# train, test = dataset.filter(
#         lambda traj, label: len(traj) >= 5
#         and traj.r.delta.norm.sum() > 0
#         and dataset.label_counts[label] > 5
#     ).split(
#         train_size=0.985,
#         random_state=SEED,
#     )

# # Build and train the model
# featurizer = featurizers.UniversalFeaturizer()
# model = XGBoostModel(featurizer=featurizer)
# # model = SVMModel(
# #     featurizer=featurizer,
# #     random_state=SEED,
# # )
# # DecisionTreeModel(
# #     featurizer=featurizer,
# #     max_depth=7,
# #     random_state=SEED,
# # # )


# # Train the model on the train dataset
# model.train(train, 5) # 5: CV

# # Evaluate the model on a test dataset
# evaluation = model.evaluate(test)

# # Print the evaluation
# evaluation.show()

### Experiment


In [13]:
from traj_xai.src import run_experiments

In [14]:
# segment_func = [rdp_segmentation]
# perturbation_func = [gaussian_perturbation]
segment_func = [sliding_window_segmentation]
perturbation_func = [scaling_perturbation, rotation_perturbation, gaussian_perturbation]

run_experiments(
    test,
    segment_func,
    perturbation_func,
    model,
)

Running experiment with sliding_window_segmentation and scaling_perturbation
Doesn't change classification
[INFO] Row saved to logs/sliding_window_segmentation_scaling_perturbation_results.csv: [0, 'traj_e274634e', 0, 0.0]
2025-07-13 09:24:48 - Total RAM: 24.00 GB | Used: 63.8%
Segments for trajectory 1: [[Vector([ 0., 97.])], [Vector([ 5., 45.])]]
[INFO] Row saved to logs/sliding_window_segmentation_scaling_perturbation_results.csv: [1, 'traj_d0f003d1', 1, 1.0]
Doesn't change classification
[INFO] Row saved to logs/sliding_window_segmentation_scaling_perturbation_results.csv: [2, 'traj_66dc088a', 0, 0.0]
Doesn't change classification
[INFO] Row saved to logs/sliding_window_segmentation_scaling_perturbation_results.csv: [3, 'traj_bce89e29', 0, 0.0]
Doesn't change classification
[INFO] Row saved to logs/sliding_window_segmentation_scaling_perturbation_results.csv: [4, 'traj_60e0114f', 0, 0.0]
Doesn't change classification
[INFO] Row saved to logs/sliding_window_segmentation_scaling_pert